In [1]:
import glob, os, subprocess
from IPython.display import IFrame

## Precursors

To train a model, you first need to convert your sequences and targets into the input HDF5 format. Check out my tutorials for how to do that; they're linked from the [main page](../README.md).

For this tutorial, grab a small example HDF5 that I constructed here with 10% of the training sequences and only GM12878 targets for various DNase-seq, ChIP-seq, and CAGE experiments.

In [2]:
if len(glob.glob('data/heart_l131k/tfrecords/*.tfr')) == 0:
    subprocess.call('curl -o data/heart_l131k.tgz https://storage.googleapis.com/basenji_tutorial_data/heart_l131k.tgz', shell=True)
    subprocess.call('tar -xzvf data/heart_l131k.tgz', shell=True)

## Train

Next, you need to decide what sort of architecture to use. This grammar probably needs work; my goal was to enable hyperparameter searches to write the parameters to file so that I could run parallel training jobs to explore the hyperparameter space. I included an example set of parameters that will work well with this data in models/params_small.txt.

Then, run [basenji_train.py](https://github.com/calico/basenji/blob/master/bin/basenji_train.py) to train a model. The program will offer training feedback via stdout and write the model output files to the prefix given by the *-s* parameter.

The most relevant options here are:

| Option/Argument | Value | Note |
|:---|:---|:---|
| -o | models/heart | Directory to save training logs and model checkpoints. |
| params_file | models/params_small.json | JSON specified parameters to setup the model architecture and optimization. |
| data_dir | data/heart_l131k | Data directory containing the test input and output datasets as generated by [basenji_data.py](https://github.com/calico/basenji/blob/master/bin/basenji_data.py) |

If you want to train, uncomment the following line and run it. Depending on your hardware, it may require several hours.

In [12]:
! basenji_train.py -o models/heart models/params_small.json data/heart_l131k

//anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
//anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
//anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
//anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1)

## Test

Alternatively, you can just download a trained model.

In [2]:
if not os.path.isdir('models/heart'):
    os.mkdir('models/heart')
if not os.path.isfile('models/heart/model_best.h5'):
    subprocess.call('curl -o models/heart/model_best.h5 https://storage.googleapis.com/basenji_tutorial_data/model_best.h5', shell=True)

models/heart/model_best.tf will now specify the name of your saved model to be provided to other programs.

To further benchmark the accuracy (e.g. computing significant "peak" accuracy), use [basenji_test.py](https://github.com/calico/basenji/blob/master/bin/basenji_test.py).

The most relevant options here are:

| Option/Argument | Value | Note |
|:---|:---|:---|
| --ai | 0,1,2 | Make accuracy scatter plots for targets 0, 1, and 2. |
| -o | output/heart_test | Output directory. |
| --rc | | Average the forward and reverse complement to form an ensemble predictor. |
| --shifts | | Average sequence shifts to form an ensemble predictor. |
| params_file | models/params_small.json | JSON specified parameters to setup the model architecture and optimization. |
| model_file | models/heart/model_best.h5 | Trained saved model parameters. |
| data_dir | data/heart_l131k | Data directory containing the test input and output datasets as generated by [basenji_data.py](https://github.com/calico/basenji/blob/master/bin/basenji_data.py) |

In [3]:
! basenji_test.py --ai 0,1,2 -o output/heart_test --rc --shifts "1,0,-1" models/params_small.json models/heart/model_best.h5 data/heart_l131k

2021-02-12 15:37:05.450758: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-02-12 15:37:05.451005: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sequence (InputLayer)           [(None, 131072, 4)]  0                                            
__________________________________________________________________________________________________
stochastic_reverse_complement ( ((None, 131072, 4),  0           sequence[0][0]                   
__________

*data/heart_test/acc.txt* is a table specifiying the Pearson correlation and R2 for each dataset. 

In [4]:
! cat output/heart_test/acc.txt

index	pearsonr	r2	identifier	description
0	0.51014	0.20690	CNhs11760	aorta
1	0.64676	0.40159	CNhs12843	artery
2	0.50528	0.21342	CNhs12856	pulmonic_valve


The directories *pr*, *roc*, *violin*, and *scatter* in *data/heart_test* contain plots for the targets indexed by 0, 1, and 2 as specified by the --ai option above.

E.g.

In [5]:
IFrame('output/heart_test/pr/t0.pdf', width=600, height=500)